In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import os
import pickle
import pandas as pd
import json
import datetime
import time
import copy

py_file_location = './drive/My Drive/PrivatePackages'
sys.path.append(os.path.abspath(py_file_location))
from sklearn.model_selection import train_test_split

import JiaoCheng
import NingXiang

In [ ]:
LABEL = 'logret'
MODEL = 'svr'

In [ ]:
curated_data = pd.read_csv('./drive/My Drive/LAB/Hackathon/data/curated_data.csv')
curated_data = curated_data.drop(columns=['closePrice', 'closePriceNorm', 'ret', 'trend', 'stock'])
curated_data['date'] = pd.to_datetime(curated_data['date'])

In [ ]:
curated_data

,date,log_ret,ret_lag_1,log_ret_lag_1,closePrice_lag_1,closePriceNorm_lag_1,ret_lag_2,log_ret_lag_2,closePrice_lag_2,closePriceNorm_lag_2,...,closePrice_lag_8,closePriceNorm_lag_8,ret_lag_9,log_ret_lag_9,closePrice_lag_9,closePriceNorm_lag_9,ret_lag_10,log_ret_lag_10,closePrice_lag_10,closePriceNorm_lag_10
0,2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-01-02,-0.002996,NaN,NaN,60.17,-0.982328,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-03,-0.005516,0.997008,-0.002996,59.99,-1.094387,NaN,NaN,60.17,-0.982328,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-04,-0.004704,0.994499,-0.005516,59.66,-1.299827,0.997008,-0.002996,59.99,-1.094387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-01-05,-0.002867,0.995307,-0.004704,59.38,-1.474140,0.994499,-0.005516,59.66,-1.299827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12495,2023-09-03,-0.005332,0.991745,-0.008289,43.25,0.244890,0.998855,-0.001146,43.61,0.508653,...,42.40,-0.377885,0.987128,-0.012955,42.18,-0.539073,0.987748,-0.012327,42.73,-0.136102
12496,2023-09-04,0.000232,0.994682,-0.005332,43.02,0.076374,0.991745,-0.008289,43.25,0.244890,...,42.42,-0.363231,1.005216,0.005202,42.40,-0.377885,0.987128,-0.012955,42.18,-0.539073
12497,2023-09-05,0.012471,1.000232,0.000232,43.03,0.083701,0.994682,-0.005332,43.02,0.076374,...,42.47,-0.326598,1.000472,0.000472,42.42,-0.363231,1.005216,0.005202,42.40,-0.377885
12498,2023-09-06,-0.010150,1.012549,0.012471,43.57,0.479346,1.000232,0.000232,43.03,0.083701,...,42.52,-0.289964,1.001179,0.001178,42.47,-0.326598,1.000472,0.000472,42.42,-0.363231


In [ ]:
date = [datetime.datetime(year=2023, month=1, day = 1) + datetime.timedelta(days=i) for i in range(250)]
val_date_start = date[175]
test_date_start = date[175+38]

In [ ]:
for i in range(1, 11):
    print('Max Lag:', i)

    tmp_curated_data = copy.deepcopy(curated_data)

    for j in range(i+1, 11):
        tmp_curated_data = tmp_curated_data.drop(columns=[f'log_ret_lag_{j}', f'ret_lag_{j}'], axis = 1)

    tmp_curated_data = tmp_curated_data.dropna()

    train = tmp_curated_data[(tmp_curated_data['date'] < val_date_start)]
    val = tmp_curated_data[(tmp_curated_data['date'] < test_date_start) & (tmp_curated_data['date'] >= val_date_start)]
    test = tmp_curated_data[(tmp_curated_data['date'] >= test_date_start)]

    jiaocheng = JiaoCheng.JiaoCheng()

    train_x = train.drop(columns=['date', 'log_ret'], axis = 1)
    train_y = train['log_ret']

    val_x = val.drop(columns=['date', 'log_ret'], axis = 1)
    val_y = val['log_ret']

    test_x = test.drop(columns=['date', 'log_ret'], axis = 1)
    test_y = test['log_ret']

    with open(f'./drive/My Drive/LAB/Hackathon/models/NingXiang_output_{LABEL}_lag_{i}.pickle', 'rb') as f:
        ningxiang_output = pickle.load(f)

    ningxiang_output = list(ningxiang_output.items())
    ningxiang_output = dict(ningxiang_output)

    jiaocheng = JiaoCheng.JiaoCheng()

    from sklearn.svm import SVR

    jiaocheng.read_in_model(SVR, 'Regression')

    jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

    del train_x
    del train_y
    del val_x
    del val_y

    parameter_choices = {
        'C': (0.0001, 0.001, 0.01, 0.1, 1, 10, 100),
        'kernel': ('poly', 'rbf'),
        'degree': (1, 2, 3)

    }

    jiaocheng.set_hyperparameters(parameter_choices)

    jiaocheng.set_non_tuneable_hyperparameters({'max_iter': -1})

    jiaocheng.set_features(ningxiang_output)

    jiaocheng.set_tuning_order(['features', 'kernel', 'degree', 'C'])

    jiaocheng.set_hyperparameter_default_values({
        'features': 0,
        'C': 1,
        'degree': 1,
        'kernel': 'poly'}
    )

    try:
        jiaocheng.read_in_tuning_result_df(f'./drive/My Drive/LAB/Hackathon/models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    except:
        pass

    jiaocheng.set_tuning_result_saving_address(f'./drive/My Drive/LAB/Hackathon/models/tuning/jiaocheng_{MODEL}_{LABEL}_lag_{i}.csv')
    jiaocheng.set_best_model_saving_address(f'./drive/My Drive/LAB/Hackathon/models/tmp_models/jiaocheng_{MODEL}_{LABEL}_lag_{i}.pickle')

    jiaocheng.tune()

    jiaocheng.view_best_combo_and_score()

    print('\n\n')

Max Lag: 1
JiaoCheng Initialised
JiaoCheng Initialised
Successfully read in model <class 'sklearn.svm._classes.SVR'>, which is a Regression model
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
Successfully recorded hyperparameter choices
Successfully recorded non_tuneable_hyperparameter choices
Successfully recorded tuneable feature combination choices and updated relevant internal structures
Successfully set tuning output address
Successfully set best model output address

Default combo: [4, 0, 0, 0] 


ROUND 1

Round 1 
Hyperparameter: features (index: 3) 

Trained and Tested combination 1 of 924: (4, 0, 0, 0), taking 0.02 seconds to get val score of -0.2998
        Current best combo: (4, 0, 0, 0) with val score -0.2998
Trained and Tested combination 2 of 924: (4, 0, 0, 1), taking 0.01 seconds to get val score of -0.2998
        Current best combo: (4, 0, 0, 0) with val score -0.2998
Trained and Tested combinat